# Bibliotecas

In [1]:
# # Instala bibliotecas usando o pip
# !pip install python-bcb # BCB
# !pip install ipeadatapy # IPEADATA
# !pip install sidrapy    # SIDRA/IBGE
# !pip install dbnomics   # OCDE e FMI (outros também)
# !pip install wbgapi     # Banco Mundial

# Importa bibliotecas/módulos
from bcb import sgs
from bcb import Expectativas
from dbnomics import fetch_series_by_api_link
from datetime import date
import pandas as pd
import ipeadatapy as ipea
import sidrapy as sidra
import wbgapi as wb

# Dados do BCB

## SGS

In [3]:
# Site: https://www3.bcb.gov.br/sgspub/
# Coletar dados da SELIC no SGS/BCB
dados_sgs = sgs.get(
    codes = 432,
    start = "2020-01-01",
    end = date.today()
)
dados_sgs

,432
Date,
2020-01-01,4.50
2020-01-02,4.50
2020-01-03,4.50
2020-01-04,4.50
2020-01-05,4.50
...,...
2022-09-22,13.75
2022-09-23,13.75
2022-09-24,13.75


In [6]:
# Coletar múltiplas séries do SGS
dados_sgs = sgs.get(
    codes = {"Dólar": 3698, "IBC-Br": 24363, "Resultado Primário": 5793},
    start = "2020-01-01",
    end = date.today()
)
dados_sgs.tail()

,Dólar,IBC-Br,Resultado Primário
Date,,,
2022-04-01,4.7580,142.40,-1.54
2022-05-01,4.9551,143.03,-1.33
2022-06-01,5.0492,142.20,-2.19
2022-07-01,5.3681,148.86,-2.50
2022-08-01,5.1411,NaN,NaN


In [8]:
# Tratamento de dados
dados_sgs.rename_axis("data").dropna().tail()

,Dólar,IBC-Br,Resultado Primário
data,,,
2022-03-01,4.9684,148.47,-1.39
2022-04-01,4.7580,142.40,-1.54
2022-05-01,4.9551,143.03,-1.33
2022-06-01,5.0492,142.20,-2.19
2022-07-01,5.3681,148.86,-2.50


In [13]:
(dados_sgs.rename_axis("data") # formato wide p/ long
    .dropna()
    .reset_index()
    .melt(
        id_vars = "data", 
        value_vars = ["Dólar", "IBC-Br", "Resultado Primário"], 
        var_name = "variavel"
        )
    .rename(columns = {"value": "valor"})
    )

,data,variavel,valor
0,2020-01-01,Dólar,4.1495
1,2020-02-01,Dólar,4.3410
2,2020-03-01,Dólar,4.8839
3,2020-04-01,Dólar,5.3256
4,2020-05-01,Dólar,5.6434
...,...,...,...
88,2022-03-01,Resultado Primário,-1.3900
89,2022-04-01,Resultado Primário,-1.5400
90,2022-05-01,Resultado Primário,-1.3300
91,2022-06-01,Resultado Primário,-2.1900


## Expectativas

In [14]:
# Site: https://www3.bcb.gov.br/expectativas2/#/consultaSeriesEstatisticas
# Coletar dados de expectativas de mercado

# Conexão com a API
exp = Expectativas().get_endpoint("ExpectativasMercadoAnuais") 

# Extração dos dados
dados_focus = (
    exp.query()
    .filter(exp.Indicador == "IPCA")
    .filter(exp.Data >= "2020-01-01")
    .collect()
    )

dados_focus

,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo
0,IPCA,None,2020-01-02,2019,4.0500,4.1100,0.2000,3.4300,4.36,121,0
1,IPCA,None,2020-01-02,2020,3.5900,3.6000,0.2500,2.4600,4.23,119,0
2,IPCA,None,2020-01-02,2021,3.7400,3.7500,0.1300,3.3000,4.10,102,0
3,IPCA,None,2020-01-02,2022,3.5600,3.5000,0.1600,3.0000,4.12,93,0
4,IPCA,None,2020-01-02,2023,3.5000,3.5000,0.2100,3.0000,4.11,78,0
...,...,...,...,...,...,...,...,...,...,...,...
6855,IPCA,None,2022-09-23,2022,5.8278,5.7744,0.3320,5.2069,6.89,88,1
6856,IPCA,None,2022-09-23,2023,4.9930,4.9987,0.4742,3.2987,7.02,87,1
6857,IPCA,None,2022-09-23,2024,3.4334,3.3000,0.4387,3.0000,5.00,75,1
6858,IPCA,None,2022-09-23,2025,3.2445,3.0000,0.4115,2.9500,5.00,64,1


In [15]:
# Extração com tratamentos de dados
dados_focus = (
    exp.query()
    .filter(exp.Indicador == "IPCA")
    .filter(exp.Data >= "2020-01-01")
    .filter(exp.baseCalculo == 0)
    .select(exp.Data, exp.DataReferencia, exp.Mediana)
    .collect()
    )

dados_focus

,Data,DataReferencia,Mediana
0,2020-01-02,2019,4.110
1,2020-01-02,2020,3.600
2,2020-01-02,2021,3.750
3,2020-01-02,2022,3.500
4,2020-01-02,2023,3.500
...,...,...,...
3425,2022-09-23,2022,5.883
3426,2022-09-23,2023,5.000
3427,2022-09-23,2024,3.500
3428,2022-09-23,2025,3.000


In [18]:
# Agregar expectativa por mês
dados_focus.rename(
    columns = {"Data": "data", "DataReferencia": "data_ref", "Mediana": "mediana"}, 
    inplace = True
    )
dados_focus

,data,data_ref,mediana
0,2020-01-02,2019,4.110
1,2020-01-02,2020,3.600
2,2020-01-02,2021,3.750
3,2020-01-02,2022,3.500
4,2020-01-02,2023,3.500
...,...,...,...
3425,2022-09-23,2022,5.883
3426,2022-09-23,2023,5.000
3427,2022-09-23,2024,3.500
3428,2022-09-23,2025,3.000


In [19]:
dados_focus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3430 entries, 0 to 3429
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      3430 non-null   object 
 1   data_ref  3430 non-null   object 
 2   mediana   3430 non-null   float64
dtypes: float64(1), object(2)
memory usage: 80.5+ KB


In [23]:
# Converter coluna data e finalizar agregação
dados_focus["data"] = pd.to_datetime(dados_focus["data"])
dados_focus["ano_mes"] = dados_focus["data"].dt.strftime("%Y/%m")
dados_focus.groupby(by = ["data_ref", "ano_mes"])["mediana"].mean()

data_ref  ano_mes
2019      2020/01    4.148333
2020      2020/01    3.543182
          2020/02    3.261667
          2020/03    3.087727
          2020/04    2.385000
                       ...   
2026      2022/05    3.000000
          2022/06    3.000000
          2022/07    3.000014
          2022/08    3.000000
          2022/09    3.000000
Name: mediana, Length: 168, dtype: float64

# Dados do IPEADATA

In [24]:
# Site: http://www.ipeadata.gov.br/
# Extrair tabela com todas séries e códigos disponíveis
series_ipeadata = ipea.metadata()
series_ipeadata.tail()

,CODE,NAME,COMMENT,LAST UPDATE,BIG THEME,SOURCE ACRONYM,SOURCE,SOURCE URL,FREQUENCY,MEASURE,UNIT,SERIES STATUS,THEME CODE,COUNTRY,NUMERICA
8923,PIBVRAE,PIB Estadual - valor adicionado a preços const...,"Fonte: IPEA, a partir de dados do IBGE. Trata-...",2021-11-25T11:32:05.457-03:00,Regional,IBGE,Instituto Brasileiro de Geografia e Estatística,www.ibge.gov.br,Anual,Porcentagem,None,None,81,None,True
8924,VPRODPIN,Valor da produção - Indústria - Total,Os municípios da tabela não necessariamente co...,2021-05-14T14:54:28-03:00,Regional,IBGE,Instituto Brasileiro de Geografia e Estatística,www.ibge.gov.br,Irregular,"R$ (mil), a preços do ano 1999",None,None,81,None,True
8925,COVID19_CASOS,Casos de COVID19 no dia,"Para os anos de 1997 a 2008 -- Fonte: IBGE, 19...",2020-06-25T17:24:03-03:00,Regional,IPEA,Instituto de Pesquisa Econômica Aplicada,www.ipea.gov.br,Diária,Pessoas,None,None,82,BRA,True
8926,COVID19_CASOSACUM,"Casos de COVID19, acumulados até o dia","Para os anos de 1997 a 2008 -- Fonte: IBGE, 19...",2020-06-25T17:24:04-03:00,Regional,IPEA,Instituto de Pesquisa Econômica Aplicada,www.ipea.gov.br,Diária,Pessoas,None,None,82,BRA,True
8927,COVID19_OBITOSACUM,"Óbitos por COVID19, acumulados até o dia","Para os anos de 1997 a 2008 -- Fonte: IBGE, 19...",2020-06-25T17:24:02-03:00,Regional,IPEA,Instituto de Pesquisa Econômica Aplicada,www.ipea.gov.br,Diária,Pessoas,None,None,82,BRA,True


In [27]:
# Filtrar séries com o termo "caged"
series_ipeadata[series_ipeadata.CODE.str.contains("caged|CAGED|Caged")]

,CODE,NAME,COMMENT,LAST UPDATE,BIG THEME,SOURCE ACRONYM,SOURCE,SOURCE URL,FREQUENCY,MEASURE,UNIT,SERIES STATUS,THEME CODE,COUNTRY,NUMERICA
6635,CAGED12_ADMIS,Empregados - admissões,Os dados referem-se ao total de admissões de ...,2020-05-27T14:48:01-03:00,Macroeconômico,MTE/Caged,"Ministério do Trabalho e Emprego, Cadastro Ger...",www.mte.gov.br,Mensal,Pessoa,None,I,12,BRA,True
6636,CAGED12_ADMISN12,Empregados - admissões - novo Caged,Para empregados consideram-se aqueles contrata...,2022-08-30T16:08:01.63-03:00,Macroeconômico,Min. Economia/SEPRT/Novo Caged,"Ministério da Economia, Secretaria Especial de...",http://pdet.mte.gov.br/novo-caged,Mensal,Pessoa,None,A,12,BRA,True
6637,CAGED12_DESLIG,Empregados - demissões,Os dados referem-se ao total de dispensa de e...,2020-05-27T14:48:00.913-03:00,Macroeconômico,MTE/Caged,"Ministério do Trabalho e Emprego, Cadastro Ger...",www.mte.gov.br,Mensal,Pessoa,None,I,12,BRA,True
6638,CAGED12_DESLIGN12,Empregados - demissões - novo Caged,Para empregados consideram-se aqueles contrata...,2022-08-30T16:08:01.63-03:00,Macroeconômico,Min. Economia/SEPRT/Novo Caged,"Ministério da Economia, Secretaria Especial de...",http://pdet.mte.gov.br/novo-caged,Mensal,Pessoa,None,A,12,BRA,True
6639,CAGED12_SALDO12,Empregados - saldo,O saldo refere-se a diferença entre o total de...,2020-05-27T14:48:00.913-03:00,Macroeconômico,MTE/Caged,"Ministério do Trabalho e Emprego, Cadastro Ger...",www.mte.gov.br,Mensal,Pessoa,None,I,12,BRA,True
6640,CAGED12_SALDON12,Empregados - saldo - novo Caged,Para empregados consideram-se aqueles contrata...,2022-08-30T16:08:01.63-03:00,Macroeconômico,Min. Economia/SEPRT/Novo Caged,"Ministério da Economia, Secretaria Especial de...",http://pdet.mte.gov.br/novo-caged,Mensal,Pessoa,None,A,12,BRA,True


In [28]:
# Coletar série de interesse usando o código
dados_ipeadata = ipea.timeseries("CAGED12_SALDON12")
dados_ipeadata.tail()

,YEAR,DAY,MONTH,CODE,RAW DATE,VALUE (Pessoa)
DATE,,,,,,
2022-03-01,2022,1,3,CAGED12_SALDON12,2022-03-01T00:00:00-03:00,136189.0
2022-04-01,2022,1,4,CAGED12_SALDON12,2022-04-01T00:00:00-03:00,196966.0
2022-05-01,2022,1,5,CAGED12_SALDON12,2022-05-01T00:00:00-03:00,277018.0
2022-06-01,2022,1,6,CAGED12_SALDON12,2022-06-01T00:00:00-03:00,277944.0
2022-07-01,2022,1,7,CAGED12_SALDON12,2022-07-01T00:00:00-03:00,218902.0


In [29]:
# Tratamento de dados
(
    dados_ipeadata.reset_index()
    .rename(columns = {"DATE": "data", "VALUE (Pessoa)": "caged"})
    .filter(items = ["data", "caged"], axis = "columns")
    ).tail()

,data,caged
26,2022-03-01,136189.0
27,2022-04-01,196966.0
28,2022-05-01,277018.0
29,2022-06-01,277944.0
30,2022-07-01,218902.0


# Dados do Sidra/IBGE

In [30]:
# Site: https://sidra.ibge.gov.br/ e https://apisidra.ibge.gov.br/
# Códigos de consulta (obtidos no site) com filtros na tabela 7060 (Sidra/IBGE)
dados_sidra = sidra.get_table(
    table_code = "7060",             # código da tabela de interesse
    territorial_level = "1",         # nível territorial
    ibge_territorial_code = "all",   # desagregações desse nível
    variable = "63",                 # variável = IPCA - Variação mensal (%)
    period = "all"                   # todas as observações
)

dados_sidra.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Mês (Código),Mês,Variável (Código),Variável,"Geral, grupo, subgrupo, item e subitem (Código)","Geral, grupo, subgrupo, item e subitem"
1,1,Brasil,2,%,0.21,1,Brasil,202001,janeiro 2020,63,IPCA - Variação mensal,7169,Índice geral
2,1,Brasil,2,%,0.25,1,Brasil,202002,fevereiro 2020,63,IPCA - Variação mensal,7169,Índice geral
3,1,Brasil,2,%,0.07,1,Brasil,202003,março 2020,63,IPCA - Variação mensal,7169,Índice geral
4,1,Brasil,2,%,-0.31,1,Brasil,202004,abril 2020,63,IPCA - Variação mensal,7169,Índice geral


In [43]:
# Tratamento de dados
(
    dados_sidra.rename(columns = dados_sidra.iloc[0])
    .rename(columns = {"Mês (Código)": "data", "Valor": "ipca"})
    .drop(labels = 0, axis = "index")
    .filter(items = ["data", "ipca"], axis = "columns")
    .assign(data = lambda x: pd.to_datetime(arg = x.data, format = "%Y%m"))
    ).tail()

,data,ipca
28,2022-04-01,1.06
29,2022-05-01,0.47
30,2022-06-01,0.67
31,2022-07-01,-0.68
32,2022-08-01,-0.36


# Dados da OECD

In [45]:
# Site: https://stats.oecd.org/
# Banco de dados utilizado: https://db.nomics.world/

# Variável de interesse: Taxa de desemprego (harmonizada)
# Conjunto de dados: Labour Force Survey
# Link da tabela dessa variável na base do DBnomics:
cod_json = (
    "https://api.db.nomics.world/v22/series/OECD/MEI?dimensions=%7B%22" +
    "SUBJECT%22%3A%5B%22LRHUTTTT%22%5D%2C%22MEASURE%22%3A%5B%22STSA%22" +
    "%5D%2C%22FREQUENCY%22%3A%5B%22M%22%5D%7D&observations=1"
    )

# Coletar dados
dados_ocde = fetch_series_by_api_link(cod_json)

dados_ocde.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,LOCATION,SUBJECT,MEASURE,FREQUENCY,Country,Subject,Measure,Frequency
0,monthly,OECD,MEI,Main Economic Indicators Publication,AUS.LRHUTTTT.STSA.M,Australia – Labour Force Survey - quarterly ra...,1978-02,1978-02-01,6.644554,6.644554,AUS,LRHUTTTT,STSA,M,Australia,Labour Force Survey - quarterly rates > Harmon...,"Level, rate or national currency, s.a.",Monthly
1,monthly,OECD,MEI,Main Economic Indicators Publication,AUS.LRHUTTTT.STSA.M,Australia – Labour Force Survey - quarterly ra...,1978-03,1978-03-01,6.301991,6.301991,AUS,LRHUTTTT,STSA,M,Australia,Labour Force Survey - quarterly rates > Harmon...,"Level, rate or national currency, s.a.",Monthly
2,monthly,OECD,MEI,Main Economic Indicators Publication,AUS.LRHUTTTT.STSA.M,Australia – Labour Force Survey - quarterly ra...,1978-04,1978-04-01,6.267243,6.267243,AUS,LRHUTTTT,STSA,M,Australia,Labour Force Survey - quarterly rates > Harmon...,"Level, rate or national currency, s.a.",Monthly
3,monthly,OECD,MEI,Main Economic Indicators Publication,AUS.LRHUTTTT.STSA.M,Australia – Labour Force Survey - quarterly ra...,1978-05,1978-05-01,6.208944,6.208944,AUS,LRHUTTTT,STSA,M,Australia,Labour Force Survey - quarterly rates > Harmon...,"Level, rate or national currency, s.a.",Monthly
4,monthly,OECD,MEI,Main Economic Indicators Publication,AUS.LRHUTTTT.STSA.M,Australia – Labour Force Survey - quarterly ra...,1978-06,1978-06-01,6.302940,6.302940,AUS,LRHUTTTT,STSA,M,Australia,Labour Force Survey - quarterly rates > Harmon...,"Level, rate or national currency, s.a.",Monthly


In [46]:
# Tratamento de dados
dados_ocde.rename(
    columns = {"period": "data", "Country": "pais", "value": "valor"}
    ).filter(items = ["data", "pais", "valor"], axis = "columns")

,data,pais,valor
0,1978-02-01,Australia,6.644554
1,1978-03-01,Australia,6.301991
2,1978-04-01,Australia,6.267243
3,1978-05-01,Australia,6.208944
4,1978-06-01,Australia,6.302940
...,...,...,...
807,2022-04-01,United States,3.600000
808,2022-05-01,United States,3.600000
809,2022-06-01,United States,3.600000
810,2022-07-01,United States,3.500000


# Dados do FMI

In [47]:
# Fonte: https://data.imf.org/
# Banco de dados utilizado: https://db.nomics.world/

# Variável de interesse: Taxa de Câmbio Real Efetiva (CPI, index)
# Conjunto de dados: International Financial Statistics (IFS)
# Link da tabela dessa variável do IFS na base do DBnomics:
cod_json = (
    "https://api.db.nomics.world/v22/series/IMF/IFS?dimensions=%7B%22INDICATOR" +
    "%22%3A%5B%22EREER_IX%22%5D%2C%22REF_AREA%22%3A%5B%22AU%22%2C%22CA%22%2C%2" +
    "2CL%22%5D%2C%22FREQ%22%3A%5B%22M%22%5D%7D&observations=1"
    )

# Coletar dados
dados_fmi = fetch_series_by_api_link(cod_json)

dados_fmi.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,FREQ,REF_AREA,INDICATOR,Frequency,Reference Area,Indicator
0,monthly,IMF,IFS,International Financial Statistics (IFS),M.AU.EREER_IX,"Monthly – Australia – Exchange Rates, Real Eff...",1979-12,1979-12-01,90.731878,90.731878,M,AU,EREER_IX,Monthly,Australia,"Exchange Rates, Real Effective Exchange Rate b..."
1,monthly,IMF,IFS,International Financial Statistics (IFS),M.AU.EREER_IX,"Monthly – Australia – Exchange Rates, Real Eff...",1980-01,1980-01-01,90.497395,90.497395,M,AU,EREER_IX,Monthly,Australia,"Exchange Rates, Real Effective Exchange Rate b..."
2,monthly,IMF,IFS,International Financial Statistics (IFS),M.AU.EREER_IX,"Monthly – Australia – Exchange Rates, Real Eff...",1980-02,1980-02-01,90.402501,90.402501,M,AU,EREER_IX,Monthly,Australia,"Exchange Rates, Real Effective Exchange Rate b..."
3,monthly,IMF,IFS,International Financial Statistics (IFS),M.AU.EREER_IX,"Monthly – Australia – Exchange Rates, Real Eff...",1980-03,1980-03-01,91.449360,91.449360,M,AU,EREER_IX,Monthly,Australia,"Exchange Rates, Real Effective Exchange Rate b..."
4,monthly,IMF,IFS,International Financial Statistics (IFS),M.AU.EREER_IX,"Monthly – Australia – Exchange Rates, Real Eff...",1980-04,1980-04-01,92.162540,92.162540,M,AU,EREER_IX,Monthly,Australia,"Exchange Rates, Real Effective Exchange Rate b..."


In [48]:
# Tratamento de dados
(
    dados_fmi.pivot(      # formato long p/ wide
        index = "period",
        columns = "Reference Area",
        values = "value"
        )
    .reset_index()    
    .dropna()
    )

Reference Area,period,Australia,Canada,Chile
11,1979-12-01,90.731878,93.336412,150.585400
12,1980-01-01,90.497395,92.845640,151.978835
13,1980-02-01,90.402501,93.745251,155.041871
14,1980-03-01,91.449360,93.151436,160.972934
15,1980-04-01,92.162540,92.400611,164.349253
...,...,...,...,...
518,2022-03-01,91.765416,84.743189,86.929155
519,2022-04-01,93.151108,85.180954,86.191073
520,2022-05-01,91.036625,84.835601,84.982310
521,2022-06-01,91.088710,85.469245,84.980786


# Dados do Banco Mundial

In [49]:
# Site: https://data.worldbank.org/
# Pesquisar códigos de variáveis sobre inflação (CPI)
wb.search(q = "inflation")

ID,Name,Field,Value
FP.CPI.TOTL,Consumer price index (2010 = 100),Developmentrelevance,...A general and continuing increase in an economy’s price level is called inflation. The increase in the average prices of goods and services in the economy should...
FP.CPI.TOTL,Consumer price index (2010 = 100),Limitationsandexceptions,"...impairing comparability over time. Although useful for measuring consumer price inflation within a country, consumer price indexes are of less value in comparing..."
FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)",IndicatorName,"Inflation, consumer prices (annual %)"
FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)",Longdefinition,...Inflation as measured by the consumer price index reflects the annual percentage change...
FR.INR.RINR,Real interest rate (%),Longdefinition,...Real interest rate is the lending interest rate adjusted for inflation as measured by the GDP deflator. The terms and conditions attached to lending...
FR.INR.RINR,Real interest rate (%),Statisticalconceptandmethodology,...interest rates are calculated by adjusting nominal rates by an estimate of the inflation rate in the economy. A negative real interest rate indicates a loss in the...
NY.GDP.DEFL.KD.ZG,"Inflation, GDP deflator (annual %)",IndicatorName,"Inflation, GDP deflator (annual %)"
NY.GDP.DEFL.KD.ZG,"Inflation, GDP deflator (annual %)",Longdefinition,...Inflation as measured by the annual growth rate of the GDP implicit deflator shows the...
NY.GDP.DEFL.KD.ZG.AD,"Inflation, GDP deflator: linked series (annual %)",IndicatorName,"Inflation, GDP deflator: linked series (annual %)"
NY.GDP.DEFL.KD.ZG.AD,"Inflation, GDP deflator: linked series (annual %)",Longdefinition,...Inflation as measured by the annual growth rate of the GDP implicit deflator shows the...


In [54]:
# Coletar dados
dados_wb = wb.data.DataFrame(
    series = "FP.CPI.TOTL.ZG",
    economy = "all", 
    time = range(2010, 2023),
    columns = "series",
    labels = True
    )
dados_wb

Country  Time  FP.CPI.TOTL.ZG
economy time                                                     
ZWE     YR2021                     Zimbabwe  2021       98.546105
        YR2020                     Zimbabwe  2020      557.201817
        YR2019                     Zimbabwe  2019      255.304991
        YR2018                     Zimbabwe  2018       10.618866
        YR2017                     Zimbabwe  2017        0.893962
...                                     ...   ...             ...
AFE     YR2014  Africa Eastern and Southern  2014        5.370290
        YR2013  Africa Eastern and Southern  2013        5.750981
        YR2012  Africa Eastern and Southern  2012        9.158707
        YR2011  Africa Eastern and Southern  2011        8.971206
        YR2010  Africa Eastern and Southern  2010        5.537538

[3192 rows x 3 columns]

In [55]:
# Tratamento de dados
(
    dados_wb.reset_index()
    .rename(
        columns = {
            "Time": "data",
            "Country": "pais",
            "economy": "codigo",
            "FP.CPI.TOTL.ZG": "cpi"
            }
            )
    .filter(items = ["data", "pais", "codigo", "cpi"], axis = "columns")
    )

,data,pais,codigo,cpi
0,2021,Zimbabwe,ZWE,98.546105
1,2020,Zimbabwe,ZWE,557.201817
2,2019,Zimbabwe,ZWE,255.304991
3,2018,Zimbabwe,ZWE,10.618866
4,2017,Zimbabwe,ZWE,0.893962
...,...,...,...,...
3187,2014,Africa Eastern and Southern,AFE,5.370290
3188,2013,Africa Eastern and Southern,AFE,5.750981
3189,2012,Africa Eastern and Southern,AFE,9.158707
3190,2011,Africa Eastern and Southern,AFE,8.971206
